In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense,Lambda
from tensorflow.keras.models import Model
from tensorflow import keras
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Load and Define Data

In [2]:
df = pd.read_csv('../Data/PUMA-1204-2012to2016-5%_Treated.csv')
df

,HINCP,NP,AGEP,RAC1P,ESR,SEX,WIF,HUPAC,HHT,PUMA,ST
0,3,4,46,1,1,2,3,2,1,1204,24
1,3,4,44,1,1,1,3,2,1,1204,24
2,3,4,13,1,0,2,3,2,1,1204,24
3,3,4,8,1,0,2,3,2,1,1204,24
4,4,3,70,2,6,1,3,4,1,1204,24
...,...,...,...,...,...,...,...,...,...,...,...
4499,4,2,73,1,6,2,1,4,1,1204,24
4500,5,4,38,1,1,1,3,2,1,1204,24
4501,5,4,36,9,1,2,3,2,1,1204,24
4502,5,4,12,1,0,1,3,2,1,1204,24


In [3]:
df=df.fillna(df.mean())
df

,HINCP,NP,AGEP,RAC1P,ESR,SEX,WIF,HUPAC,HHT,PUMA,ST
0,3,4,46,1,1,2,3,2,1,1204,24
1,3,4,44,1,1,1,3,2,1,1204,24
2,3,4,13,1,0,2,3,2,1,1204,24
3,3,4,8,1,0,2,3,2,1,1204,24
4,4,3,70,2,6,1,3,4,1,1204,24
...,...,...,...,...,...,...,...,...,...,...,...
4499,4,2,73,1,6,2,1,4,1,1204,24
4500,5,4,38,1,1,1,3,2,1,1204,24
4501,5,4,36,9,1,2,3,2,1,1204,24
4502,5,4,12,1,0,1,3,2,1,1204,24


In [4]:
df.shape

(4504, 11)

# Data Preprocessing

In [5]:
X_train,X_test=train_test_split(df,test_size=200)


In [6]:
X_train

,HINCP,NP,AGEP,RAC1P,ESR,SEX,WIF,HUPAC,HHT,PUMA,ST
841,6,4,54,1,1,1,4,4,1,1204,24
2151,5,5,9,1,0,2,3,2,1,1204,24
2368,5,3,67,1,1,1,3,4,1,1204,24
1070,5,6,10,1,0,2,3,2,1,1204,24
2054,6,3,37,1,1,1,3,1,1,1204,24
...,...,...,...,...,...,...,...,...,...,...,...
2539,2,2,60,1,1,2,3,4,1,1204,24
113,3,4,29,9,1,2,4,4,2,1204,24
1065,3,3,11,1,0,2,3,2,1,1204,24
1023,5,6,39,1,1,1,2,3,2,1204,24


In [7]:
minmax=MinMaxScaler()
X_train = minmax.fit_transform(X_train)
X_test = minmax.fit_transform(X_test)

In [8]:
X_train

array([[1.        , 0.25      , 0.57446809, ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 0.375     , 0.09574468, ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 0.125     , 0.71276596, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.25      , 0.125     , 0.11702128, ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 0.5       , 0.41489362, ..., 0.5       , 0.        ,
        0.        ],
       [0.75      , 0.375     , 0.18085106, ..., 0.        , 0.        ,
        0.        ]])

In [9]:
def fit_batchsize(X,batch_size):
    n_size = (len(X)//batch_size)*batch_size
    X = X[0:n_size]

    return X
batch_size = 10
X_train = fit_batchsize(X_train, batch_size)
X_test = fit_batchsize(X_test, batch_size)

# Define VAE Network

In [10]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [11]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(11,))
x = layers.Reshape((11,1))(encoder_inputs)
x = layers.Conv1D(12,3, activation="relu", strides=1, padding="same")(x)
x = layers.Conv1D(24,3,activation="relu", strides=1, padding="same")(x)
x = layers.Flatten()(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 11)]         0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 11, 1)        0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 11, 12)       48          reshape[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 11, 24)       888         conv1d[0][0]                     
____________________________________________________________________________________________

In [12]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(11 * 24, activation="relu")(latent_inputs)
x = layers.Reshape((11, 24))(x)
x = layers.Conv1DTranspose(24, 3, activation="relu", strides=1, padding="same")(x)
x = layers.Conv1DTranspose(12, 3, activation="relu", strides=1, padding="same")(x)
x = layers.Conv1DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder_outputs=layers.Reshape((11,))(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 264)               792       
_________________________________________________________________
reshape_1 (Reshape)          (None, 11, 24)            0         
_________________________________________________________________
conv1d_transpose (Conv1DTran (None, 11, 24)            1752      
_________________________________________________________________
conv1d_transpose_1 (Conv1DTr (None, 11, 12)            876       
_________________________________________________________________
conv1d_transpose_2 (Conv1DTr (None, 11, 1)             37        
_________________________________________________________________
reshape_2 (Reshape)          (None, 11)                0   

In [13]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 11
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

# Training the VAE Network

In [14]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history=vae.fit(X_train,
        X_train, 
        epochs=20, 
        batch_size=10)

Epoch 1/20
430/430 [==============================] - 1s 2ms/step - loss: 5.3724 - reconstruction_loss: 5.1779 - kl_loss: 0.1946
Epoch 2/20
430/430 [==============================] - 1s 2ms/step - loss: 4.7635 - reconstruction_loss: 4.1703 - kl_loss: 0.5932
Epoch 3/20
430/430 [==============================] - 1s 2ms/step - loss: 4.7394 - reconstruction_loss: 4.0883 - kl_loss: 0.6511
Epoch 4/20
430/430 [==============================] - 1s 2ms/step - loss: 4.7289 - reconstruction_loss: 4.0443 - kl_loss: 0.6846
Epoch 5/20
430/430 [==============================] - 1s 2ms/step - loss: 4.6851 - reconstruction_loss: 3.9727 - kl_loss: 0.7124
Epoch 6/20
430/430 [==============================] - 1s 2ms/step - loss: 4.6848 - reconstruction_loss: 3.9309 - kl_loss: 0.7539
Epoch 7/20
430/430 [==============================] - 1s 2ms/step - loss: 4.6474 - reconstruction_loss: 3.8486 - kl_loss: 0.7988
Epoch 8/20
430/430 [==============================] - 1s 2ms/step - loss: 4.6144 - reconstruction

# Reconstruct and Augment Data

In [15]:
z_mean, z_log_var, z = encoder(X_test)
reconstruction=decoder(z)
X_test=tf.convert_to_tensor(X_test)

fin_X_test=X_test

for i in range(100):
    z_mean, z_log_var, z = encoder(X_test)
    reconstruction=tf.concat([reconstruction,decoder(z)],0)

print(reconstruction)


tf.Tensor(
[[7.1650082e-01 2.8602290e-01 2.9280430e-01 ... 3.1124949e-03
  1.5145540e-04 2.7617812e-04]
 [5.0334191e-01 2.1349937e-02 7.3934877e-01 ... 1.4912099e-02
  3.3959031e-07 2.3514345e-05]
 [5.0849789e-01 4.5859426e-02 6.6553962e-01 ... 2.1167409e-01
  4.3673322e-06 5.0954099e-05]
 ...
 [6.6221052e-01 8.0469966e-02 5.4494619e-01 ... 7.7068508e-03
  5.5282510e-07 7.0403279e-05]
 [6.8082643e-01 2.1205494e-01 3.8130698e-01 ... 2.1486998e-02
  1.8766522e-04 1.4936924e-04]
 [7.4892652e-01 2.7496231e-01 2.6154357e-01 ... 3.3646822e-04
  3.7949994e-05 2.7826428e-04]], shape=(20200, 11), dtype=float32)


In [16]:
reconstruction=minmax.inverse_transform(reconstruction)
reconstruction

array([[4.86600327e+00, 3.43011451e+00, 2.75236043e+01, ...,
        1.00622499e+00, 1.20400015e+03, 2.40002762e+01],
       [4.01336765e+00, 2.10674968e+00, 6.94987843e+01, ...,
        1.02982420e+00, 1.20400000e+03, 2.40000235e+01],
       [4.03399158e+00, 2.22929713e+00, 6.25607245e+01, ...,
        1.42334819e+00, 1.20400000e+03, 2.40000510e+01],
       ...,
       [4.64884210e+00, 2.40234983e+00, 5.12249422e+01, ...,
        1.01541370e+00, 1.20400000e+03, 2.40000704e+01],
       [4.72330570e+00, 3.06027469e+00, 3.58428558e+01, ...,
        1.04297400e+00, 1.20400019e+03, 2.40001494e+01],
       [4.99570608e+00, 3.37481153e+00, 2.45850958e+01, ...,
        1.00067294e+00, 1.20400004e+03, 2.40002783e+01]])

In [17]:
reconstruction=np.round(reconstruction)
reconstruction

array([[5.000e+00, 3.000e+00, 2.800e+01, ..., 1.000e+00, 1.204e+03,
        2.400e+01],
       [4.000e+00, 2.000e+00, 6.900e+01, ..., 1.000e+00, 1.204e+03,
        2.400e+01],
       [4.000e+00, 2.000e+00, 6.300e+01, ..., 1.000e+00, 1.204e+03,
        2.400e+01],
       ...,
       [5.000e+00, 2.000e+00, 5.100e+01, ..., 1.000e+00, 1.204e+03,
        2.400e+01],
       [5.000e+00, 3.000e+00, 3.600e+01, ..., 1.000e+00, 1.204e+03,
        2.400e+01],
       [5.000e+00, 3.000e+00, 2.500e+01, ..., 1.000e+00, 1.204e+03,
        2.400e+01]])

In [18]:
reconstruction=pd.DataFrame(reconstruction)

In [19]:
reconstruction.to_csv('reconstruction PUMA==1204.csv')